# Feature Engineering - 0629_08-10-35 (consider all features)

### Overview
This notebook serves a simple utility purpose to take a single data log with all network signals considered as features and does the initial preprocessing to cleaned & raw so it can be used by the LSTM modeling notebook. 

In this case we take the initial coverted raw csv file from the blf conversion contained in /can_data/*.csv and perform LSTM preprocessing and store to the  /can_data/lstm_processed folder. The file name can be changed to perform this lstm preprocessing on any other raw network data csv file by simply changing the filename below. 

LSTM Preprocessing 
- Load data frame log
- Set time as index and downsample to 1s
- Drop bad signals
- Clean column names to just include signal name
- Backfill NaN data
- Elimnate duplicate columns

In [5]:
# Import all the necessary libraries to be used by this script

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pathlib import Path

from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.metrics import confusion_matrix, roc_curve, auc, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Resolve folder paths before reading data
RAW_CSV_DIR = Path("can_data")
PREPROCESSED_CSV_DIR = Path("can_data/lstm_preprocessed") 

In [6]:
# Load cleaned  data from 0629_35 - 1.5 hours, 1.5 cycles, IBS signals only

path = RAW_CSV_DIR / "12V Management  Discharge and Charge Cycle 2025-06-29_08-10-35.csv"
path_cleaned_raw = PREPROCESSED_CSV_DIR / "12V Management  Discharge and Charge Cycle 2025-06-29_08-10-35_cleaned_raw.csv"

# Most efficient: parse the datetime column on read and set as the index
df = pd.read_csv(
    path,
    parse_dates=["time_utc_iso"],    # let pandas parse ISO8601 (handles 'Z' -> UTC)
)

# Ensure timezone-aware UTC (if the parsed datetimes are naive)
if df["time_utc_iso"].dt.tz is None:
    # If strings had a trailing 'Z', pandas usually sets UTC automatically; otherwise, localize:
    df["time_utc_iso"] = df["time_utc_iso"].dt.tz_localize("UTC")

# Set as index and sort
df = df.set_index("time_utc_iso").sort_index()

df = df.drop(columns=["time_et_iso", "excel_utc"], errors="ignore") # drop not needed time columns

print(df.index[:5])
print(df.index.dtype)   # should show datetime64[ns, UTC]
print(df.shape)


/var/folders/ng/3vcxzr395fd86ffyyn36q09w0000gn/T/ipykernel_14067/1205057653.py:7: DtypeWarning: Columns (27,30,31,32,33,35,36,108,109,112,114,115,116,117,118,120,121,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,168,169,172,174,175,176,177,178,179,181,182,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,237,434,479,480,603,604,605,606,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,696,697,698,699,700,701,702,703,704,705,707,854,1078,1079,1082,1083,1084,1085,1177,1179,1180,1181,1182,1183,1243,1245,1247,1249,1250,1251,1252,1253,1254,1255,1257,1258,1260,1

DatetimeIndex(['2025-06-29 12:10:35+00:00', '2025-06-29 12:10:36+00:00',
               '2025-06-29 12:10:37+00:00', '2025-06-29 12:10:38+00:00',
               '2025-06-29 12:10:39+00:00'],
              dtype='datetime64[ns, UTC]', name='time_utc_iso', freq=None)
datetime64[ns, UTC]
(4734, 3078)


In [7]:
df.head()

,VCU_0x214.VCU_214_234_CheckSum,VCU_0x214.VCU_RemWakeUpEndFlg,VCU_0x214.VCU_BrkLampCtrlSts,VCU_0x214.VCU_RdyLamp,VCU_0x214.VCU_214_234_AliveCounter,VCU_0x214.VCU_GearSigVld,VCU_0x214.VCU_ShiftMisoper,VCU_0x214.VCU_StgyGearSig,VCU_0x214.VCU_APSPercVld,VCU_0x214.VCU_ParkRdy,...,Diag_OBC_Resp.OBC_Byte7_Resp,Diag_OBC_Resp.OBC_Byte8_Resp,Diag_PVIU_Resp.PVIU_TPCI_Resp,Diag_PVIU_Resp.PVIU_Byte2_Resp,Diag_PVIU_Resp.PVIU_Byte3_Resp,Diag_PVIU_Resp.PVIU_Byte4_Resp,Diag_PVIU_Resp.PVIU_Byte5_Resp,Diag_PVIU_Resp.PVIU_Byte6_Resp,Diag_PVIU_Resp.PVIU_Byte7_Resp,Diag_PVIU_Resp.PVIU_Byte8_Resp
time_utc_iso,,,,,,,,,,,,,,,,,,,,,
2025-06-29 12:10:35+00:00,176,remote_control_not_ended,No_request,Ready,0,Valid,Normal,gear_P,Valid,Not_ready,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-29 12:10:36+00:00,216,remote_control_not_ended,No_request,Ready,10,Valid,Normal,gear_P,Valid,Not_ready,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-29 12:10:37+00:00,132,remote_control_not_ended,No_request,Ready,5,Valid,Normal,gear_P,Valid,Not_ready,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-29 12:10:38+00:00,176,remote_control_not_ended,No_request,Ready,0,Valid,Normal,gear_P,Valid,Not_ready,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-29 12:10:39+00:00,216,remote_control_not_ended,No_request,Ready,10,Valid,Normal,gear_P,Valid,Not_ready,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Downsample if not already 1 second

# 1) Numeric columns → median per second
df_num = df.resample("1S").median(numeric_only=True)

# 2) The rest (non-numeric) → take the last value seen each second, then forward-fill gaps
non_num_cols = df.columns.difference(df_num.columns)
df_non = df[non_num_cols].resample("1S").last().ffill()

# 3) Recombine and keep as your working df
df = pd.concat([df_num, df_non], axis=1).sort_index()

print(df.index[:5])
print(df.shape)


/var/folders/ng/3vcxzr395fd86ffyyn36q09w0000gn/T/ipykernel_14067/2224660911.py:4: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df_num = df.resample("1S").median(numeric_only=True)
/var/folders/ng/3vcxzr395fd86ffyyn36q09w0000gn/T/ipykernel_14067/2224660911.py:8: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df_non = df[non_num_cols].resample("1S").last().ffill()


DatetimeIndex(['2025-06-29 12:10:35+00:00', '2025-06-29 12:10:36+00:00',
               '2025-06-29 12:10:37+00:00', '2025-06-29 12:10:38+00:00',
               '2025-06-29 12:10:39+00:00'],
              dtype='datetime64[ns, UTC]', name='time_utc_iso', freq='s')
(4734, 3078)


In [9]:
# Drop bad signals

# drop constant cols
df = df.loc[:, df.nunique() > 1]

# drop mostly empty cols
missing_thresh = 0.5
df = df.loc[:, df.isnull().mean() < missing_thresh]

# drop checksums and alivecounters
df = df.drop(columns=df.filter(like="CheckSum").columns)
df = df.drop(columns=df.filter(like="AliveCounter").columns)
df = df.drop(columns=df.filter(like="BMS_CellVolt").columns)

print(df.shape)

(4734, 408)


In [10]:
# Shorten column names to remove the CAN message and only keep the signal name
new_columns = []
for col in df.columns:
    if '.' in col:
        name = col.split('.', 1)[1]
    else:
        name = col
    new_columns.append(name)

df.columns = new_columns 
print(df.columns)

Index(['MCU_F_CrtSpd', 'MCU_F_CrtTq', 'MCU_F_MaxPwrGennTq', 'MCU_F_MaxElecTq',
       'MCU_F_MOT_T', 'MCU_F_IGBTT', 'MCU_R_CrtSpd', 'MCU_R_CrtTq',
       'MCU_R_MaxPwrGennTq', 'MCU_R_MaxElecTq',
       ...
       'VCU_VehMod', 'VCU_LVbattCtrlSigSTGFlt', 'VCU_HVBActivateDeactivateRq',
       'VCU_Set_BMS_Mod', 'IBS_CurrentAutorange', 'VSP_HornWngSts',
       'VSP_WngMod', 'VSP_UsrData1_Bit6_DIAGN_AWAKE', 'WTC_B_HVPwrSplySts',
       'WTC_H_HVPwrSplySts'],
      dtype='object', length=408)


In [11]:
# Handle NaNs 
df = df.interpolate(method="linear").ffill().bfill()

print(df.shape)

/var/folders/ng/3vcxzr395fd86ffyyn36q09w0000gn/T/ipykernel_14067/4210384042.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.interpolate(method="linear").ffill().bfill()


(4734, 408)


In [12]:
# Eliminate duplicate columns
dups = df.columns[df.columns.duplicated()]
print(f"Duplicate columns: {len(dups)}")
if len(dups):
    print(dups.unique()[:20])  # peek

Duplicate columns: 10
Index(['GW_CtrlBitVector_Bit0_RMR', 'GW_UsrData1_Bit5_IGNITION_AWAKE',
       'PKC_IMMO_AuthSts', 'PKC_KeySts', 'PKC_RemLockCmd_Key', 'TBOX_4GSigIND',
       'VCU_RdyLamp', 'VCU_VehSt', 'VCU_ChrgSysOperCmd', 'VCU_VehChrgDchgMod'],
      dtype='object')


In [13]:
# Save a raw version for later for SOH1 calculation
df_raw = df.copy()

### Ready to save preprocessed dataframe
- Load data frame log
- Set time as index and downsample to 1s
- Drop bad signals
- Clean column names to just include signal name
- Backfill NaN data
- Elimnate duplicate columns

In [14]:
# Save raw preprocessed data before converting non-numeric (one-hot encoding), scaling, & soh1 calculation
# Save cleaned data file for next steps
df.to_csv(path_cleaned_raw)

In [15]:
df.shape

(4734, 408)